# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.26,74,96,12.62,TF,1698041699
1,1,axim,4.8699,-2.2405,25.26,91,96,3.07,GH,1698041700
2,2,blackmans bay,-43.0167,147.3167,18.62,62,68,1.34,AU,1698041700
3,3,puerto ayora,-0.7393,-90.3518,23.17,84,85,4.49,EC,1698041700
4,4,waitangi,-43.9535,-176.5597,8.79,77,85,0.45,NZ,1698041569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:


# Configure the map plot
coordinates_df = city_data_df[["Lat","Lng","City","Humidity"]]

# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City"
)
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.dropna()
city_data_df2 = city_data_df2[(city_data_df2['Max Temp']>=21) & (city_data_df2['Max Temp']<=27)]


# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,axim,4.8699,-2.2405,25.26,91,96,3.07,GH,1698041700
3,3,puerto ayora,-0.7393,-90.3518,23.17,84,85,4.49,EC,1698041700
6,6,samba,32.5667,75.1167,26.49,36,0,1.59,IN,1698041701
7,7,salinopolis,-0.6136,-47.3561,26.87,81,64,5.49,BR,1698041701
10,10,labasa,-16.4167,179.3833,25.52,84,95,4.17,FJ,1698041702
...,...,...,...,...,...,...,...,...,...,...
567,567,cabedelo,-6.9811,-34.8339,23.37,80,99,4.92,BR,1698041738
570,570,ghadamis,30.1337,9.5007,26.27,22,0,3.54,LY,1698041827
573,573,la foa,-21.7108,165.8276,24.73,59,68,4.42,NC,1698041828
581,581,dourados,-22.2211,-54.8056,23.20,45,37,2.95,BR,1698041830


### Step 3: Create a new DataFrame called `hotel_df`.

In [57]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df2[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,['Hotel Name']] = ''
hotel_df
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,axim,GH,4.8699,-2.2405,91,
3,puerto ayora,EC,-0.7393,-90.3518,84,
6,samba,IN,32.5667,75.1167,36,
7,salinopolis,BR,-0.6136,-47.3561,81,
10,labasa,FJ,-16.4167,179.3833,84,
...,...,...,...,...,...,...
567,cabedelo,BR,-6.9811,-34.8339,80,
570,ghadamis,LY,30.1337,9.5007,22,
573,la foa,NC,-21.7108,165.8276,59,
581,dourados,BR,-22.2211,-54.8056,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000
filters = ''
bias = ''
params = {
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = row['Lng']
    Lat = row['Lat']    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
        # Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
axim - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
samba - nearest hotel: No hotel found
salinopolis - nearest hotel: No hotel found
labasa - nearest hotel: No hotel found
dakhla - nearest hotel: No hotel found
nueva esperanza - nearest hotel: No hotel found
anna regina - nearest hotel: No hotel found
banikoara - nearest hotel: No hotel found
brisas barra de suchiate - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
ghanzi - nearest hotel: No hotel found
ghormach - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
peshawar - nearest hotel: No hotel found
alofi - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
tucurui - nearest hotel: No hotel found
baiti - nearest hotel: No hotel found
muisne - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
bouake - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
1,axim,GH,4.8699,-2.2405,91,No hotel found
3,puerto ayora,EC,-0.7393,-90.3518,84,No hotel found
6,samba,IN,32.5667,75.1167,36,No hotel found
7,salinopolis,BR,-0.6136,-47.3561,81,No hotel found
10,labasa,FJ,-16.4167,179.3833,84,No hotel found
...,...,...,...,...,...,...
567,cabedelo,BR,-6.9811,-34.8339,80,No hotel found
570,ghadamis,LY,30.1337,9.5007,22,No hotel found
573,la foa,NC,-21.7108,165.8276,59,No hotel found
581,dourados,BR,-22.2211,-54.8056,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
# Configure the map plot
coordinates_df = hotel_df[["Lat","Lng","City","Humidity","Hotel Name", "Country"]]

# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City",
    hover_cols = "all"
)
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Hotel Name,Country)